In [1]:
import numpy as np
import os
import io
import cv2

import torch

import sys
PATH_ROOT=os.path.realpath(os.path.join(os.curdir, '..'))
sys.path.append(PATH_ROOT)
print(PATH_ROOT)


/opt/src/intern_video/InternVideo2/multi_modality


In [2]:
for m in ['demo_config', 'demo_utils']:
    if m in sys.modules:
        del sys.modules[m]

In [3]:

from demo_config import (Config,
                    eval_dict_leaf)

from demo_utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)


[2024-05-21 16:40:22,420] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
video = cv2.VideoCapture('example1.mp4')
frames = [x for x in _frame_from_video(video)]

In [5]:
text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
                   "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
                   "A person dressed in a blue jacket shovels the snow-covered pavement outside their house.",
                   "A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner.",
                   "A person stands on the snowy floor, pushing a sled loaded with blankets, preparing for a fun-filled ride.",
                   "A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees.",
                   "A playful dog slides down a snowy hill, wagging its tail with delight.",
                   "A person in a blue jacket walks their pet on a leash, enjoying a peaceful winter walk among the trees.",
                   "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
                   "A person bundled up in a blanket walks through the snowy landscape, enjoying the serene winter scenery."]

In [6]:
config = Config.from_file('internvideo2_stage2_config.py')
config = eval_dict_leaf(config)

In [7]:
model_pth = '/cwd/weights/pretrained/InternVideo2-stage2_1b-224p-f4.pt'
config['pretrained_path'] = model_pth
config['model']['vision_encoder']['pretrained']='/cwd/weights/pretrained/InternVideo2-stage2_1b-224p-f4.pt'
config['model']['text_encoder']['config']='../configs/config_bert_large.json'

In [8]:
intern_model, tokenizer = setup_internvideo2(config)

/opt/conda/envs/py310/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


bert tokenizer loaded
vision encoder built
text encoder built
intern video2 stage2 loaded
sending model to cuda
model sent to cuda
loading model from /cwd/weights/pretrained/InternVideo2-stage2_1b-224p-f4.pt
interpolating position embedding.. 
load_state_dict: _IncompatibleKeys(missing_keys=[], unexpected_keys=['temp', 'itm_head.weight', 'itm_head.bias'])
model type conversion..
model type converted


In [9]:
intern_model.eval()
intern_model.cuda()
print(1)

1


In [10]:
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=5, config=config)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

torch.Size([1, 4, 3, 224, 224])
text: A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run. ~ prob: 0.7927
text: A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon. ~ prob: 0.1769
text: A pet dog excitedly runs through the snowy yard, chasing a toy thrown by its owner. ~ prob: 0.0291
text: A man in a gray hat and coat walks through the snowy yard, carefully navigating around the trees. ~ prob: 0.0006
text: A person dressed in a blue jacket shovels the snow-covered pavement outside their house. ~ prob: 0.0003


In [11]:
import torch_tensorrt

import torch
from torch import nn

In [12]:
class VideoEncoder(nn.Module):
    def __init__(self, vlm: nn.Module):
        super().__init__()
        self.vlm = vlm

    def forward(self, frames: torch.Tensor):
        _, vfeat = self.vlm.encode_vision(frames, test=True)
        vfeat = self.vlm.vision_proj(vfeat)
        vfeat /= vfeat.norm(dim=-1, keepdim=True)
        return vfeat

video_encoder = VideoEncoder(intern_model)
video_encoder.eval()
input_data = torch.empty([1, 4, 3, 224, 224]).cuda()
traced_video_encoder = torch.jit.trace(video_encoder, input_data)
# print(traced_video_encoder)

In [13]:
print(traced_video_encoder.__class__)

<class 'torch.jit._trace.TopLevelTracedModule'>


In [17]:
frozen_traced_video_encoder = torch.jit.optimize_for_inference(traced_video_encoder)
torch.jit.save(traced_video_encoder, '/cwd/video_encoder.pt')

In [22]:
#trt_model = torch_tensorrt.compile(frozen_traced_video_encoder,
#    inputs= [torch_tensorrt.Input((1, 4, 3, 224, 224))],
#    enabled_precisions= { torch.half }, # Run with FP32
#    truncate_long_and_double=True,
#)
frozen_traced_video_encoder(torch.zeros(1,4,3,224,224).cuda())

tensor([[ 0.0716,  0.0473, -0.0347,  0.0136,  0.0743,  0.0895, -0.0271,  0.0458,
         -0.0419,  0.0484, -0.0265, -0.0260, -0.0154, -0.0332,  0.0334, -0.0044,
         -0.1580,  0.0580,  0.1798,  0.0084, -0.0402, -0.0150, -0.0090, -0.0078,
         -0.0389, -0.0400, -0.0363,  0.0075,  0.0159, -0.0059,  0.0341,  0.0359,
         -0.0045, -0.0185, -0.0415, -0.0027,  0.0067,  0.0658, -0.0512,  0.0339,
         -0.0149, -0.0072, -0.0648,  0.0451,  0.0149, -0.0085, -0.0425,  0.0435,
         -0.0036,  0.0202,  0.0028,  0.0214, -0.0170,  0.0375, -0.0155,  0.0951,
         -0.0441, -0.0542, -0.0451,  0.0061,  0.0083, -0.0012, -0.0908,  0.0134,
          0.0589,  0.0200, -0.0386,  0.0039, -0.0962,  0.0124,  0.0089, -0.0414,
          0.0713,  0.0161, -0.0447, -0.0038, -0.0113, -0.0551,  0.1673, -0.0665,
         -0.0295,  0.0309, -0.0558,  0.0033, -0.0371, -0.0204,  0.0160, -0.0069,
         -0.0563, -0.0793, -0.0061,  0.0077, -0.0218, -0.0442,  0.0552, -0.0440,
         -0.0338, -0.0133, -